In [ ]:
import funzioni02 as f2
from copy import deepcopy
import numpy as np
import datetime
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

In [ ]:
shots=8192
reps=4        # al massimo 10
precision=40
time=np.pi
backend_aus=backend_sim_jakarta
#check=["4copy_check", [0,2,4,6]]
evo_type="column_evolution" # "column_evolution" or "complete_evolution" or "trotter_steps"
measure_mitigation = "yes"  #"yes" or "no"
steps=42

In [ ]:
qcs_reps = []

for _ in range(reps):
    check=["copy_check", [0,2,4]]
    qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
    qcs_reps=qcs_reps + qcs
    
    check=["4copy_check", [0,2,4,6]]
    qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
    qcs_reps=qcs_reps + qcs

reps=reps*2

In [ ]:
check=["copy_check", [0,2,4]]
qcs_calibs, meas_calibs = f2.calibration_cirquits(evo_type+"_remake", q_anc=check[1], check="yes", check_type=check[0])
state_lables = f2.bin_list(3+len(check[1]))

check=["4copy_check", [0,2,4,6]]
qcs_calibs4, meas_calibs4 = f2.calibration_cirquits(evo_type+"_remake", q_anc=check[1], check="yes", check_type=check[0])
state_lables4 = f2.bin_list(3+len(check[1])) 

In [ ]:
job_evo=execute(qcs_reps, backend=backend_aus, shots=shots)

job_cal=execute(meas_calibs, backend=backend_aus, shots=shots)
job_cal_our=execute(qcs_calibs, backend=backend_aus, shots=shots)

job_cal4=execute(meas_calibs4, backend=backend_aus, shots=shots)
job_cal_our4=execute(qcs_calibs4, backend=backend_aus, shots=shots)

jobs_evo_res=[]

In [ ]:
for i in range(reps):
    
    if i%2==0:
        job=execute(qcs_reps[0:27],backend=backend_sim, shots=10)
    else:
        job=execute(qcs_reps[27:27*2],backend=backend_sim, shots=10)
        
    results = job.result()
    
    for j in range(27):
        results.results[j].data.counts = job_evo.result().get_counts()[i*27+j]
    
    #print(results.get_counts()[0])
    jobs_evo_res.append(results)

In [ ]:
meas_fitter = mc.CompleteMeasFitter(job_cal.result(), state_labels=state_lables)
meas_fitter_our = mc.CompleteMeasFitter(job_cal_our.result(), state_labels=state_lables)
meas_fitter4 = mc.CompleteMeasFitter(job_cal4.result(), state_labels=state_lables4)
meas_fitter_our4 = mc.CompleteMeasFitter(job_cal_our4.result(), state_labels=state_lables4)

target_state = (One^One^Zero).to_matrix()

fids=np.zeros([4])

fids=np.zeros([reps,4])
fids_mean=np.zeros(4)
fids_dev=np.zeros(4)

fids4=np.zeros([4])

fids4=np.zeros([reps,4])
fids_mean4=np.zeros(4)
fids_dev4=np.zeros(4)

ii=0
iy=0
for i in range(len(jobs_evo_res)):

    print(i)
    if i%2==0:
        
        res=deepcopy(jobs_evo_res[i])

        new_res, new_res_nm = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['011','110','101'], meas_fitter=meas_fitter)
        new_res_our, new_res_nm = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['011','110','101'], meas_fitter=meas_fitter_our)
        new_res_not_mitigated = f2.mitigate(res, Measure_Mitig="no", ancillas_conditions=f2.bin_list(3))
        
        fids[ii,0] = f2.fidelity_count(new_res_not_mitigated, qcs_na, target_state)
        fids[ii,1] = f2.fidelity_count(new_res_nm, qcs_na, target_state)
        fids[ii,2] = f2.fidelity_count(new_res, qcs_na, target_state)
        fids[ii,3] = f2.fidelity_count(new_res_our, qcs_na, target_state)

        ii+=1

    else:

        res=deepcopy(jobs_evo_res[i])

        new_res4, new_res_nm4 = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter4)
        new_res_our4, new_res_nm4 = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_our4)
        new_res_not_mitigated4 = f2.mitigate(res, Measure_Mitig="no", ancillas_conditions=f2.bin_list(4))

        
        fids4[iy,0] = f2.fidelity_count(new_res_not_mitigated4, qcs_na, target_state)
        fids4[iy,1] = f2.fidelity_count(new_res_nm4, qcs_na, target_state)
        fids4[iy,2] = f2.fidelity_count(new_res4, qcs_na, target_state)
        fids4[iy,3] = f2.fidelity_count(new_res_our4, qcs_na, target_state)

        iy+=1

for i in range(4):
    fids_mean[i]=np.mean(fids[:,i])
    fids_dev[i]=np.std(fids[:,i])

    fids_mean4[i]=np.mean(fids4[:,i])
    fids_dev4[i]=np.std(fids4[:,i])

In [ ]:
for i in range(4):
    print("no: ", fids_mean[i], "+-" ,fids_dev[i])
    print("yes: ",fids_mean4[i], "+-" ,fids_dev4[i])

In [ ]:
fids

In [ ]:
fids4